In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("/content/drive/MyDrive/convertcsv.csv")
df.head()


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   age               28 non-null     int64
 1   affordibility     28 non-null     int64
 2   bought_insurance  28 non-null     int64
dtypes: int64(3)
memory usage: 800.0 bytes


In [4]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)
X_train.shape

(22, 2)

In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [10]:
X_train_scaled.head()

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


In [11]:
model = keras.Sequential(
    [
        keras.layers.Dense(1,input_shape=(2,),activation = 'sigmoid',kernel_initializer='ones',bias_initializer = 'zeros')
    ]
)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
Epoch 1067/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5973 - accuracy: 0.6818
Epoch 1068/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5972 - accuracy: 0.6818
Epoch 1069/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5972 - accuracy: 0.6818
Epoch 1070/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5971 - accuracy: 0.6818
Epoch 1071/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5971 - accuracy: 0.6818
Epoch 1072/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5970 - accuracy: 0.6818
Epoch 1073/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5970 - accuracy: 0.6818
Epoch 1074/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5969 - accuracy: 0.6818
Epoch 1075/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5969 - accuracy: 0.6818
Epoch 1076/5000
1/1 [============

In [12]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 176ms/step - loss: 0.3550 - accuracy: 1.0000


[0.3549776077270508, 1.0]

In [20]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [13]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 112ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)

In [22]:



ag = int(input("Enter your age:\n"))
ag = ag / 100
af = int(input("Enter your affordability (0/1):\n"))

newpred = pd.DataFrame({'Age': [ag], 'Affordability': [af]})


predictions = model.predict(newpred)

if predictions>=0.5:
  print("You will buy the insurance!")
else:
  print("You will Not buy the insurance!")
print(predictions)


Enter your age:
60
Enter your affordability (0/1):
1
1/1 [==============================] - 0s 32ms/step
You will buy the insurance!
[[0.82222086]]


In [24]:
weights, bias = model.get_weights()

weights, bias

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

In [29]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

In [30]:
def prediction_function(age, affordibility):
    weighted_sum = weights[0]*age + weights[1]*affordibility + bias
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.705484819775958

In [31]:
prediction_function(.18,1)

0.35569571637887243

# **GRADIENT DESCENT FUNCTION**

In [32]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [42]:
def gradient_descent_funct(age,affordibility,y_t,epochs,loss_threshold):

  w1 = w2 =1
  bias=0

  learning_rate=0.5

  n = len(age)

  for i in range(epochs):
    weight_sum = w1 * age + w2 * affordibility + bias
    y_pred = sigmoid_numpy(weight_sum)


    loss = log_loss(y_t,y_pred)

    w1d = (1/n)*(np.dot(np.transpose(age) , (y_pred-y_t)))
    w2d = (1/n)*(np.dot(np.transpose(affordibility) , (y_pred-y_t)))

    bias_d = np.mean(y_pred-y_t)

    w1 = w1 - learning_rate * w1d
    w2 = w2 - learning_rate * w2d

    bias = bias - learning_rate * bias_d

    print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
    if loss<=loss_threshold:
            break
  return w1,w2,bias









In [43]:
gradient_descent_funct(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4650)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.681264778737757
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:0.6591474252715025
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:0.6431523291301917
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:0.6316873063379158
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:0.623471707997592
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:0.6175321183044205
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:0.6131591858705934
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:0.6098518179750948
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bia

(4.95107153085077, 1.453666829143015, -2.9159475189195407)

In [45]:
w1,w2,bias = gradient_descent_funct(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4650)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.681264778737757
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:0.6591474252715025
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:0.6431523291301917
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:0.6316873063379158
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:0.623471707997592
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:0.6175321183044205
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:0.6131591858705934
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:0.6098518179750948
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bia

In [63]:
class myNN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def fit(self,X,y,epochs,loss_threshold):

    self.w1 ,self.w2, self.bias = self.gradient_descent_funct(X['age'],X['affordibility'],y,epochs,loss_threshold)


  def predict(self,X_test):

    weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias

    return sigmoid_numpy(weighted_sum)



  def gradient_descent_funct(self,age,affordibility,y_t,epochs,loss_threshold):

    w1 = w2 =1
    bias=0

    learning_rate=0.67

    n = len(age)

    for i in range(epochs):
      weight_sum = w1 * age + w2 * affordibility + bias
      y_pred = sigmoid_numpy(weight_sum)


      loss = log_loss(y_t,y_pred)

      w1d = (1/n)*(np.dot(np.transpose(age) , (y_pred-y_t)))
      w2d = (1/n)*(np.dot(np.transpose(affordibility) , (y_pred-y_t)))

      bias_d = np.mean(y_pred-y_t)

      w1 = w1 - learning_rate * w1d
      w2 = w2 - learning_rate * w2d

      bias = bias - learning_rate * bias_d
      if i%50==0:
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
      if loss<=loss_threshold:
              break
    return w1,w2,bias





In [65]:
CSKmodel = myNN()
CSKmodel.fit(X_train_scaled,y_train,epochs = 500,loss_threshold = 0.4620)

Epoch:0, w1:0.9663762288500372, w2:0.9307864880286687, bias:-0.151981027667339, loss:0.7113403233723417
Epoch:50, w1:1.7497737158560227, w2:1.1942116709147805, bias:-1.3985596353126057, loss:0.5566824880326382
Epoch:100, w1:2.6528041204911648, w2:1.3530361440604932, bias:-1.8922124690456061, loss:0.5239363270731711
Epoch:150, w1:3.459002458516112, w2:1.405146716322588, bias:-2.2644312828799307, loss:0.5001686319385906
Epoch:200, w1:4.1685879543447975, w2:1.42977230651906, bias:-2.5755006496779203, loss:0.4821326643441314
Epoch:250, w1:4.79216495194495, w2:1.4486059221733572, bias:-2.8466428650863707, loss:0.4682481134118165


In [66]:
CSKmodel.predict(X_test_scaled)

2     0.705807
10    0.352903
21    0.160223
11    0.476150
14    0.726567
9     0.830654
dtype: float64

In [67]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 30ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)